In [1]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import matplotlib as plt
from scipy.stats import hypergeom

In [2]:
tsv = 'shCD44_vs_shLuc_all.tsv'
df_sh = pd.read_csv(tsv, sep='\t')
tsv = 'Genes_glyc.xlsx'
df_gl = pd.read_excel(tsv)

In [3]:
df_diff = df_sh.loc[(df_sh['padj']<0.05) & (abs(df_sh['log2FoldChange'])>=np.log2(1.5))]
df_diff = df_diff.loc[df_diff["Type"] == "protein_coding"]

In [4]:
tsv = 'EPAS1_targets_TFlink.tsv'
df_targets = pd.read_csv(tsv, sep = '\t')
df_targets.rename(columns={'Name.Target':'Name_Target'}, inplace=True)   
df_targets.head()

,UniprotID.Target,NCBI.GeneID.Target,Name_Target,Detection.method,PubmedID,Organism,Source.database,Small-scale.evidence
0,Q03692,1300,COL10A1,inferred by curator,"29087512,20495570",Homo sapiens,TRRUST,Yes
1,P15692,7422,VEGFA,"chromatin immunoprecipitation assay,inferred b...","11301389,11751212,22458515,24234451,27899662,2...",Homo sapiens,"GTRD,HTRI,IntAct_via_DoRothEA,KEGG_via_DoRothE...",Yes
2,P35968,3791,KDR,inferred by curator,17202159,Homo sapiens,TRED,Yes
3,P05121,5054,SERPINE1,"chromatin immunoprecipitation assay,electropho...","29126285,27924024,15039136,17202159,29087512,2...",Homo sapiens,"GTRD,HTRI,ReMap,TRED,TRRUST",Yes
4,Q9Y6K9,8517,IKBKG,inferred by curator,17202159,Homo sapiens,TRED,Yes


In [5]:
classes = df_gl.iloc[:, 1].unique()
N = len(df_diff.index.unique())
M = len(df_sh.loc[(df_sh['Type']=='protein_coding')])

In [6]:
n_arr = []
k_arr = []
p_value = []
intersec = []
for i in classes:
    df_temp = df_gl.loc[(df_gl['Class']==i)]
    n = len(df_temp.iloc[:, 0].unique())
    genes1 = df_diff.iloc[:, 0].unique()
    genes2 = df_temp.iloc[:, 0]
    inter = set(genes1) & set(genes2)
    k = len(set(genes1) & set(genes2))
    
    p_v = ss.hypergeom.sf(k, M, n, N)
    p_value.append(p_v)
    n_arr.append(n)
    k_arr.append(k)
    intersec.append(inter)

In [7]:
M_arr = [M]*len(classes)
N_arr = [N]*len(classes)

In [8]:
final_df = pd.DataFrame({'Class': classes, 'Total protein coding genes': M_arr, 'Total diff genes': N_arr, 
                         'Genes in class': n_arr, 'Genes': intersec, 'p_value': p_value})
final_df = final_df.set_index('Class')
final_df['padj'] = np.minimum(final_df['p_value'] * len(final_df) / ss.rankdata(final_df['p_value']), 1)
final_df = final_df.sort_values('padj')
final_df.head()

,Total protein coding genes,Total diff genes,Genes in class,Genes,p_value,padj
Class,,,,,,
O-glycan_mucin-type,19924,4214,35,"{ST3GAL2, GALNT10, GCNT1, GALNT2, GALNT5, GALN...",0.008730,0.161508
UDP-GalNAc_polypeptide,19924,4214,22,"{GALNT10, GALNT2, GALNT5, GALNT8, GALNT7, GALN...",0.028361,0.262344
"EC_class_2,4,99",19924,4214,12,"{ST3GAL2, ST6GALNAC2, ST3GAL1, ST3GAL5, ST6GAL1}",0.025276,0.311738
Glycolipid_globo_series,19924,4214,10,"{ST3GAL2, FUT1, B3GALT5, ST3GAL1, A4GALT}",0.008483,0.313878
Galactosyltransferases_ceramide,19924,4214,15,"{B3GALT5, B4GALT5, B4GALT6, B4GALT1, A4GALT}",0.077000,0.569800


In [50]:
final_df.to_csv('Glycosylated_genes_diff.csv')

In [47]:
# final_df = final_df.rename(index={'Glycolipid_lacto/neolacto_series': 'Glycolipid_lacto_neolacto_series'})

In [48]:
i = 0
for name in final_df.index.tolist():
    l = final_df.iloc[i, 3]
    n = final_df.index[i]
    if len(l) != 0:
        diff_genes = df_diff[df_diff['Gene'].str.contains('|'.join(l))]
        i+=1
        diff_genes.to_csv(n+'.csv')
    else:
        i+=1

In [ ]:
n = len(df_targets.iloc[:, 2].unique())
diff_targets = df_targets[df_targets['Name_Target'].isin(df_diff.iloc[:, 0])]
k = len(diff_targets)
hypergeom.sf(M, n, N, k)

In [ ]:
diff_targets.to_csv('diff_targets.csv')